In [16]:
# DOwnload the onnx model from hugging face (configs too)
#  Then downgrade a version because the used triton version does not support the ONNX IR version 10

import onnx

# Load the ONNX model
model_path = "/workspace/onnx-nomic-embed-v1.5/model.onnx"
onnx_model = onnx.load(model_path)

# Set IR version to 9
onnx_model.ir_version = 9

# Save the updated model
onnx.save(onnx_model, "/workspace/onnx-nomic-embed-v1.5/model_ir9.onnx")

print("Successfully downgraded ONNX IR version to 9!")


Successfully downgraded ONNX IR version to 9!


In [ ]:
import onnxruntime as ort
import numpy as np
import torch
import torch.nn.functional as F
from transformers import AutoImageProcessor
from PIL import Image
import requests

# Load the image processor using the preprocessor config
processor = AutoImageProcessor.from_pretrained("/workspace/onnx-nomic-embed-v1.5")

# Load the ONNX model
onnx_model_path = "/workspace/onnx-nomic-embed-v1.5/model.onnx"
session = ort.InferenceSession(onnx_model_path)

# Load an image
url = 'https://m.media-amazon.com/images/I/61sADwl+YWL._AC_UL320_.jpg'
image = Image.open(requests.get(url, stream=True).raw)

# Preprocess the image
inputs = processor(image, return_tensors="np")
onnx_inputs = {session.get_inputs()[0].name: inputs["pixel_values"]}

# Run inference
onnx_outputs = session.run(None, onnx_inputs)

# Extract and normalize embeddings
img_emb = torch.tensor(onnx_outputs[0])
img_embeddings = F.normalize(img_emb[:, 0], p=2, dim=1)

# print("Image Embeddings:", img_embeddings.shape)
img_embeddings